# Pretrained Network

## VGG16

In [1]:
from tensorflow.keras.applications import VGG16

# 기학습된 모델
# 16: Convolution Layer의 개수
model_base = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(150,150,3))

model_base.summary() # Activation Map(FC Layer의 입력 데이터): (None, 4, 4, 512) 

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
______________________________________________________________

## Cats & Dogs Training Data Set에 대한 Activation Map 추출

In [8]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 경로 설정
base_dir = '/content/drive/MyDrive/ML Colab/data/CAT_DOG/cat_dog_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# ImageDataGenerator 생성 (데이터 증식 제외)
datagen = ImageDataGenerator(rescale=1/255)

# 배치 사이즈 설정 (ImageDataGenerator - 무한히 반복하여 데이터를 추출)
batch_size = 20

# 입력 이미지에 대해 VGG16을 적용하여 특징을 추출하는 함수 생성 (->Activation Map)
# 개와 고양이 폴더를 합친 총 데이터의 개수인 sample_count중에서 batch_size개씩을 이미지를 가져옴
def extract_feature(directory, sample_count): # 입력 데이터 폴더, 데이터 개수
    
    # 최종 결과물 features: Activation Map(Numpy Array)
		# 기학습된 모델에 배치 사이즈만큼 입력 데이터를 넣어서 나온 결과를 features에 추가해줄 것이다.
    features = np.zeros(shape=(sample_count, 4,4,512)) 

    # 분류값(1차원: feature 이미지 하나 당 값 1개로 표현)
    labels = np.zeros(shape=(sample_count,)) 

    # 특정 폴더의 이미지를 가져와서 generator 생성
    generator = datagen.flow_from_directory(
          directory, # 이미지 폴더
          target_size=(150,150)
          batch_size=batch_size # 1epoch 당 가져올 데이터의 개수
          class_mode='binary'
    )

    # 이미지로부터 픽셀 데이터와 레이블을 generator의 batch_size만큼 가져옴
    i = 0
    for x_data_batch, t_data_batch in generator:
      # Convolution layer, Pooling layer에 이미지 입력
      feature_batch = model_base.predict(x_data_batch) # 최종 예측값이 나와야하지만 Classifier를 제외했으므로 Activation Map이 나옴
      features[i*batch_size:(i+1)*batch_size] = feature_batch # 행단위로 추가됨
      labels[i*batch_size:(i+1)*batch_size] = t_data_batch

      i += 1
      
      if i*batch_size >= sample_count:
        break

    return features, labels

# Feature Extraction 실행
train_features, train_labels = extract_feature(train_dir, 2000)
validation_features, validation_labels = extract_feature(validation_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Classifier
Activation Map을 FC Layer의 입력 데이터로 사용

In [9]:
# 2차원 데이터로 변경
train_features = np.reshape(train_features, (2000, 4*4*512))
validation_features = np.reshape(validation_features, (1000, 4*4*512))

# DNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 모델 생성
model = Sequential()

# FC Layer
model.add(Dense(256, activation='relu', input_shape=(4*4*512,)))

# Dropout
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(1, activation='sigmoid'))

# Optimizer
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# learning
history = model.fit(train_features, train_labels,
                    epochs=30, batch_size=64, 
                    validation_data=(validation_features, validation_labels))

Epoch 1/30
32/32 [==============================] - 1s 16ms/step - loss: 0.6212 - accuracy: 0.6624 - val_loss: 0.3485 - val_accuracy: 0.8630
Epoch 2/30
32/32 [==============================] - 0s 6ms/step - loss: 0.3411 - accuracy: 0.8499 - val_loss: 0.2916 - val_accuracy: 0.8920
Epoch 3/30
32/32 [==============================] - 0s 6ms/step - loss: 0.2837 - accuracy: 0.8894 - val_loss: 0.2661 - val_accuracy: 0.8970
Epoch 4/30
32/32 [==============================] - 0s 6ms/step - loss: 0.2267 - accuracy: 0.9169 - val_loss: 0.2595 - val_accuracy: 0.8990
Epoch 5/30
32/32 [==============================] - 0s 6ms/step - loss: 0.1941 - accuracy: 0.9306 - val_loss: 0.2439 - val_accuracy: 0.9070
Epoch 6/30
32/32 [==============================] - 0s 6ms/step - loss: 0.1952 - accuracy: 0.9295 - val_loss: 0.2360 - val_accuracy: 0.9090
Epoch 7/30
32/32 [==============================] - 0s 6ms/step - loss: 0.1682 - accuracy: 0.9391 - val_loss: 0.2360 - val_accuracy: 0.9070
Epoch 8/30
32/32 [=